# 7. 실전타임: 한국시장에서 버핏류 주식을 찾아보자

[7강의](https://www.youtube.com/watch?v=klnhcMJJS-s) 를 파이썬으로 구현해보겠습니다.

## 설치

Pandas 가 있어야 합니다. 설치가 안되어있으시면 다음 셀을 실행시키세요.

In [1]:
import sys
!{sys.executable} -m pip install pandas xlrd openpyxl

## 다운로드

[네이버 카페 퀀트킹(QuantKing)](https://cafe.naver.com/quantking)에서 최신 엑셀 파일을 다운받으세요.

저는 **QuantKing(2019.01.25).zip** 파일을 받았습니다. 받으면 확장자 **xlsb** 파일이 보입니다. 엑셀로 여셔서 **QuantKing.xlsx** 파일로 다시 저장해주세요.

## 파일 로딩

파일을 열어서 조사해 보겠습니다. Pandas의 read_excel() 이란 함수를 이용하면 열 수 있네요.

In [2]:
import pandas as pd

In [3]:
filepath = 'QuantKing.xlsx'
df = pd.read_excel(filepath, sheet_name='퀀트데이타', skiprows=2).drop('Unnamed: 0', axis=1)
df.head()

,코드 번호,회사명,업종 (대),업종 (소),코스피코스닥,주가 (원),시가총액 (억),상장주식수 (만주),자사주 (만주),자사주 비중 (%),...,18년 2Q.2,18년 3Q.2,18년 4Q(E).2,19년 1Q(E).2,18년4Q 매출액,18년4Q 영업이익,18년4Q 순이익,18년4Q 매출액.1,18년4Q 영업이익.1,18년4Q 순이익.1
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,44750,2671478,596978.0,0,0.0,...,109815,129674,100441.00,79168.0,590000.00,108000.00,NaN,631283.0,133354.0,100441.0
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,74600,543090,72800.0,4400,6.0,...,43299,46936,33962.90,16797.0,99380.81,44300.75,33962.90,101549.0,50660.0,40819.0
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,128500,274564,21367.0,939,4.4,...,7006,2692,-1297.74,7829.0,256695.34,5011.27,-1297.74,261108.0,7611.0,6727.0
3,A068270,셀트리온,제약 및 바이오,바이오,코스피,208500,266772,12795.0,85,0.7,...,840,562,544.00,521.0,NaN,NaN,NaN,2512.0,792.0,544.0
4,A207940,삼성바이오로직스,제약 및 바이오,바이오,코스피,403000,266645,6617.0,0,0.0,...,-96,-301,4362.00,28.0,NaN,NaN,NaN,1393.0,149.0,4362.0


강의에서 사용하는 컬럼만 잘라서 보관하도록 하겠습니다.

In [4]:
df= df[['회사명',
        '시가총액\n(억)', '발표\nPBR', '과거\nPER', '과거\nPSR', '시가\n배당률\n(%)', # 저평가
        '과거\nROE\n(%)', '14년->17년\n3년간 YOY', '단순\n부채비율\n(%)', '배당\n성향\n(%)', # 우량주
        '주가\n변동성', # 변동성
       ]].set_index('회사명')
df.head()

,시가총액 (억),발표 PBR,과거 PER,과거 PSR,시가 배당률 (%),과거 ROE (%),14년->17년 3년간 YOY,단순 부채비율 (%),배당 성향 (%),주가 변동성
회사명,,,,,,,,,,
삼성전자,2671478,1.14,5.62,1.07,1.9,20.3,16.2,39,12.3,0.01720
SK하이닉스,543090,1.16,3.53,1.37,1.3,35.7,75.8,32,6.4,0.02344
현대차,274564,0.40,10.28,0.29,3.1,2.2,8.0,140,20.3,0.01940
셀트리온,266772,10.53,78.90,26.31,0.0,13.3,101.5,31,0.6,0.03486
삼성바이오로직스,266645,7.13,-249.20,50.91,0.0,-2.9,340.8,95,0.0,0.03688


## 저평가 순위

저평가 순위를 매겨보겠습니다.

In [5]:
df = df[df['발표\nPBR'] != 0]
df = df[df['과거\nPER'] != 0]
df = df[df['과거\nPSR'] != 0]

df['1/PBR'] = 1 / df['발표\nPBR']
df['1/PER'] = 1 / df['과거\nPER']
df['1/PSR'] = 1 / df['과거\nPSR']

df['순위 배당수익'] = df['시가\n배당률\n(%)'].rank(ascending=False)
df['순위 PBR'] = df['1/PBR'].rank(ascending=False)
df['순위 PER'] = df['1/PER'].rank(ascending=False)
df['순위 PSR'] = df['1/PSR'].rank(ascending=False)
df['통합 저평가 지표'] = df[['순위 배당수익', '순위 PBR', '순위 PER', '순위 PSR']].mean(axis=1)

df.sort_values(by='통합 저평가 지표').head()

,시가총액 (억),발표 PBR,과거 PER,과거 PSR,시가 배당률 (%),과거 ROE (%),14년->17년 3년간 YOY,단순 부채비율 (%),배당 성향 (%),주가 변동성,1/PBR,1/PER,1/PSR,순위 배당수익,순위 PBR,순위 PER,순위 PSR,통합 저평가 지표
회사명,,,,,,,,,,,,,,,,,,
세아제강지주,2195,0.28,1.40,0.09,3.4,20.3,-6.7,82,27.9,0.02971,3.571429,0.714286,11.111111,156.5,23.5,10.0,30.5,55.125
대성홀딩스,1030,0.25,4.70,0.11,3.9,5.3,-28.9,87,19.1,0.01096,4.000000,0.212766,9.090909,104.5,17.5,93.5,60.0,68.875
E1,4157,0.33,3.30,0.09,3.3,10.0,-36.1,183,16.3,0.01398,3.030303,0.303030,11.111111,172.0,57.5,34.0,30.5,73.500
SK디스커버리,5312,0.37,0.74,0.07,2.7,50.2,-7.4,151,1.6,0.01857,2.702703,1.351351,14.285714,256.0,95.5,4.0,15.5,92.750
HDC,10305,0.54,1.11,0.19,5.8,49.1,19.7,64,14.0,0.03242,1.851852,0.900901,5.263158,26.0,266.5,8.0,157.0,114.375


## 우량주 순위

우량주 순위를 매겨보겠습니다.

In [6]:
df['ROE 순위'] = df['과거\nROE\n(%)'].rank(ascending=False)
df['영업이익 성장 순위'] = df['14년->17년\n3년간 YOY'].rank(ascending=False)
df['부채비율 순위'] = df['단순\n부채비율\n(%)'].rank(ascending=True)
df['배당성향 순위'] = df['배당\n성향\n(%)'].rank(ascending=False)
df['통합 우량주 지표'] = df[['ROE 순위', '영업이익 성장 순위', '부채비율 순위', '배당성향 순위']].mean(axis=1)

df.sort_values(by='통합 우량주 지표').head()

,시가총액 (억),발표 PBR,과거 PER,과거 PSR,시가 배당률 (%),과거 ROE (%),14년->17년 3년간 YOY,단순 부채비율 (%),배당 성향 (%),주가 변동성,...,순위 배당수익,순위 PBR,순위 PER,순위 PSR,통합 저평가 지표,ROE 순위,영업이익 성장 순위,부채비율 순위,배당성향 순위,통합 우량주 지표
회사명,,,,,,,,,,,,,,,,,,,,,
MP한강,912,2.62,14.25,1.48,11.8,18.4,209.2,7,156.7,0.04675,...,3.0,1648.0,604.0,1307.0,890.500,136.0,100.0,58.5,23.0,79.375
에이티넘인베스트,1063,1.38,8.24,3.50,2.7,16.7,158.8,5,33.1,0.04492,...,256.0,1174.5,290.0,1739.5,865.000,187.5,145.0,35.5,300.5,167.125
케어젠,7370,3.03,24.32,11.53,1.7,12.4,102.4,5,50.1,0.02518,...,499.0,1724.5,883.5,1941.0,1262.000,324.0,234.5,35.5,160.0,188.500
이크레더블,1921,4.02,15.62,5.21,3.3,25.7,69.7,18,58.5,0.00928,...,172.0,1838.0,669.0,1834.0,1128.250,57.0,361.5,276.0,118.0,203.125
메카로,1634,1.20,5.12,1.50,3.7,23.5,351.1,10,18.6,0.02977,...,124.0,1055.5,118.5,1313.5,652.875,72.0,48.0,112.5,589.5,205.500


## 변동성 순위

변동성 순위를 매겨보겠습니다.

In [7]:
df['변동성 순위'] = df['주가\n변동성'].rank(ascending=True)
df.sort_values(by='변동성 순위').head()

,시가총액 (억),발표 PBR,과거 PER,과거 PSR,시가 배당률 (%),과거 ROE (%),14년->17년 3년간 YOY,단순 부채비율 (%),배당 성향 (%),주가 변동성,...,순위 PBR,순위 PER,순위 PSR,통합 저평가 지표,ROE 순위,영업이익 성장 순위,부채비율 순위,배당성향 순위,통합 우량주 지표,변동성 순위
회사명,,,,,,,,,,,,,,,,,,,,,
UCI,484,4.48,-16.13,2.47,0.0,-27.8,-21.5,75,0.0,0.0,...,1870.0,1644.0,1591.5,1664.625,1831.0,1732.5,1064.5,1450.5,1519.625,3.5
MP그룹,1063,2.52,-10.42,0.83,0.0,-24.2,0.8,50,0.0,0.0,...,1627.5,1721.0,929.5,1457.750,1810.0,1271.0,753.0,1450.5,1321.125,3.5
케이에스피,518,8.63,-6.64,3.05,0.0,-130.0,-51.8,662,0.0,0.0,...,1972.0,1809.0,1690.5,1756.125,1997.0,1940.0,1951.0,1450.5,1834.625,3.5
썬텍,352,1.85,39.11,2.84,0.0,4.7,-24.4,66,0.0,0.0,...,1419.5,1045.0,1667.5,1421.250,918.0,1764.0,958.5,1450.5,1272.750,3.5
마제스타,672,1.93,-11.39,10.03,0.0,-16.9,-82.4,22,0.0,0.0,...,1445.0,1708.0,1929.0,1658.750,1747.5,2003.0,344.5,1450.5,1386.375,3.5


## 통합 순위

통합 순위를 매겨보겠습니다. 단순히 (저평가 지표 * 0.4 + 우량주지표 * 0.4 + 변동성 지표 * 0.2) 로 계산하시고 정렬하시면 됩니다.

In [8]:
df['통합순위'] = df['통합 저평가 지표'] * 0.4 + df['통합 우량주 지표'] * 0.4 + df['변동성 순위'] * 0.2
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
df.sort_values(by='통합순위').head()

,시가총액 (억),발표 PBR,과거 PER,과거 PSR,시가 배당률 (%),과거 ROE (%),14년->17년 3년간 YOY,단순 부채비율 (%),배당 성향 (%),주가 변동성,1/PBR,1/PER,1/PSR,순위 배당수익,순위 PBR,순위 PER,순위 PSR,통합 저평가 지표,ROE 순위,영업이익 성장 순위,부채비율 순위,배당성향 순위,통합 우량주 지표,변동성 순위,통합순위
회사명,,,,,,,,,,,,,,,,,,,,,,,,,
동원개발,3878,0.59,4.07,0.75,3.7,14.5,65.5,19,14.4,0.01467,1.694915,0.245700,1.333333,124.0,335.5,57.0,860.5,344.250,240.5,390.0,295.0,685.5,402.750,108.0,320.40
에스에이엠티,1640,0.79,4.75,0.14,6.7,16.6,23.3,63,36.9,0.01606,1.265823,0.210526,7.142857,16.0,611.0,97.0,95.0,204.750,190.5,777.5,920.0,258.0,536.500,157.0,327.90
화성산업,1911,0.51,3.70,0.39,5.8,13.8,35.0,67,23.4,0.01658,1.960784,0.270270,2.564103,26.0,237.0,46.5,456.5,191.500,260.5,615.5,968.5,464.0,577.125,177.5,342.95
KB오토시스,631,0.71,5.79,0.40,4.6,12.2,42.3,64,23.4,0.01351,1.408451,0.172712,2.500000,62.0,501.5,162.0,471.0,299.125,336.0,538.5,933.5,464.0,568.000,75.5,361.95
싸이맥스,857,0.82,4.90,0.56,3.1,16.8,200.0,26,14.5,0.02009,1.219512,0.204082,1.785714,197.5,646.5,104.0,672.0,405.000,183.5,108.0,412.0,680.5,346.000,349.5,370.30


강의의 결과와 비슷한 결과를 얻습니다. 강의에서 1등인 에스에이엠티는 여기서는 2등. 강의에서 4등인 동원개발이 여기에서는 1등이네요.